In [1]:
import pandas as pd
import numpy as np
!pip install tqdm
from tqdm import tqdm_notebook as tqdm
from itertools import islice

## Read data and combine both human and GPT dataframes 

In [2]:
def read_data(file_path, author):
    data = []

    with open(file_path, 'r') as file:
        for line_number, line_content in enumerate(file, start=1):
            row_data = []
            row_data.append('<START> ')
            for char in line_content.strip():
                # Remove unwanted chars
                if char == '.' or char == ',' or char == '!' or char == '?' or (char.isalpha() or char.isdigit() or char.isspace()):
                    row_data.append(char)
            row_data.append(' <END>')
            row_data = [element.lower() for element in row_data]
            
            # To ensure each row only has one column
            if len(row_data) > 0:
                data.append([''.join(row_data), author])

    return pd.DataFrame(data, columns=['Content', 'Author'])

# To view whole line
pd.set_option('display.max_colwidth', None)

# Test
hum = read_data("hum.txt", 'Human')
print("Dimensions of dataframe containing human generated documents:", hum.shape)

gpt = read_data("gpt.txt", 'GPT')
print("Dimensions of dataframe containing gpt generated documents:", gpt.shape)

df = pd.concat([hum, gpt], ignore_index = True)
print("Dimensions of dataframe containing both document classes:", df.shape)

Dimensions of dataframe containing human generated documents: (39713, 2)
Dimensions of dataframe containing gpt generated documents: (20398, 2)
Dimensions of dataframe containing both document classes: (60111, 2)


## Split data into train and test

In [3]:
hum_train_amount = round(0.9 * hum.shape[0])
gpt_train_amount = round(0.9 * gpt.shape[0])

# Create training and testing dataframes
columns = ['Content', 'Author']
train = pd.DataFrame(columns = columns)
test = pd.DataFrame(columns = columns)
train_hum = pd.DataFrame(columns = columns)
test_hum = pd.DataFrame(columns = columns)
train_gpt = pd.DataFrame(columns = columns)
test_gpt = pd.DataFrame(columns = columns)

train_hum = hum.iloc[:hum_train_amount]
train_gpt = gpt.iloc[:gpt_train_amount]

test_hum = hum.iloc[hum_train_amount:].reset_index(drop=True)
test_gpt = gpt.iloc[gpt_train_amount:].reset_index(drop=True)

train = pd.concat([train_hum, train_gpt], ignore_index=True)
test = pd.concat([test_hum, test_gpt], ignore_index=True)

print("Dimensions of training dataframe:", train.shape)
print("Dimensions of testing dataframe:", test.shape)

#print(test_hum['Content'][0])
#print(test_hum.head())

Dimensions of training dataframe: (54100, 2)
Dimensions of testing dataframe: (6011, 2)


## Get n-grams per class in the training corpus

In [4]:
# Function to find ngrams
def find_ngrams(words, n):
    ngrams = []
    for i in range(len(words) - n + 1):
        ngram = tuple(words[i : i + n])
        if ngram != ('<end>', '<start>'):  # Relationship between documents (?)
            ngrams.append(ngram)
    return ngrams

Find bi/trigrams per document in human and gpt training sets 
- train_bigrams_hum_dict 
- train_trigrams_hum_dict
- train_bigrams_gpt_dict
- train_trigrams_gpt_dict

In [10]:
# Tokenize data 
train_words_hum = train_hum.copy()
train_words_hum['Content'] = train_words_hum['Content'].apply(lambda x: x.split())
train_words_gpt = train_gpt.copy()
train_words_gpt['Content'] = train_words_gpt['Content'].apply(lambda x: x.split())

# Find n-grams 
train_bigrams_hum = train_words_hum['Content'].apply(lambda x: find_ngrams(x, 2))
train_trigrams_hum = train_words_hum['Content'].apply(lambda x: find_ngrams(x, 3))
train_bigrams_hum_dict = train_bigrams_hum.to_dict()
train_trigrams_hum_dict = train_trigrams_hum.to_dict()

train_bigrams_gpt = train_words_gpt['Content'].apply(lambda x: find_ngrams(x, 2))
train_trigrams_gpt = train_words_gpt['Content'].apply(lambda x: find_ngrams(x, 3))
train_bigrams_gpt_dict = train_bigrams_gpt.to_dict()
train_trigrams_gpt_dict = train_trigrams_gpt.to_dict()

for key, trigrams_list in train_trigrams_hum_dict.items():
    if len(trigrams_list) >= 2:
        second_word = trigrams_list[1][0]
        trigrams_list.insert(0, ('<start>', '<start>', second_word))
for key, trigrams_list in train_trigrams_gpt_dict.items():
    if len(trigrams_list) >= 2:
        second_word = trigrams_list[1][0]
        trigrams_list.insert(0, ('<start>', '<start>', second_word))


#print("train_trigrams_gpt_dict:")
#print(list(train_trigrams_gpt_dict.items())[:3])

train_trigrams_gpt_dict:
[(0, [('<start>', '<start>', 'when'), ('<start>', 'when', 'a'), ('when', 'a', 'breeze'), ('a', 'breeze', 'blows'), ('breeze', 'blows', 'on'), ('blows', 'on', 'your'), ('on', 'your', 'skin'), ('your', 'skin', ','), ('skin', ',', 'it'), (',', 'it', 'can'), ('it', 'can', 'help'), ('can', 'help', 'to'), ('help', 'to', 'evaporate'), ('to', 'evaporate', 'sweat'), ('evaporate', 'sweat', 'from'), ('sweat', 'from', 'your'), ('from', 'your', 'body'), ('your', 'body', '.'), ('body', '.', 'this'), ('.', 'this', 'process'), ('this', 'process', 'of'), ('process', 'of', 'evaporation'), ('of', 'evaporation', 'cools'), ('evaporation', 'cools', 'your'), ('cools', 'your', 'skin'), ('your', 'skin', ','), ('skin', ',', 'which'), (',', 'which', 'is'), ('which', 'is', 'why'), ('is', 'why', 'a'), ('why', 'a', 'breeze'), ('a', 'breeze', 'can'), ('breeze', 'can', 'feel'), ('can', 'feel', 'cool'), ('feel', 'cool', ','), ('cool', ',', 'even'), (',', 'even', 'if'), ('even', 'if', 'the'), (

## Get n-grams per class in the testing corpus

Find bi/trigrams per document in human and gpt testing sets 
- test_bigrams_hum_dict 
- test_trigrams_hum_dict
- test_bigrams_gpt_dict
- test_trigrams_gpt_dict

In [11]:
# Tokenize data 
test_words_hum = test_hum.copy()
test_words_hum['Content'] = test_words_hum['Content'].apply(lambda x: x.split())
test_words_gpt = test_gpt.copy()
test_words_gpt['Content'] = test_words_gpt['Content'].apply(lambda x: x.split())

# Find n-grams 
test_bigrams_hum = test_words_hum['Content'].apply(lambda x: find_ngrams(x, 2))
test_trigrams_hum = test_words_hum['Content'].apply(lambda x: find_ngrams(x, 3))
test_bigrams_hum_dict = test_bigrams_hum.to_dict()
test_trigrams_hum_dict = test_trigrams_hum.to_dict()

test_bigrams_gpt = test_words_gpt['Content'].apply(lambda x: find_ngrams(x, 2))
test_trigrams_gpt = test_words_gpt['Content'].apply(lambda x: find_ngrams(x, 3))
test_bigrams_gpt_dict = test_bigrams_gpt.to_dict()
test_trigrams_gpt_dict = test_trigrams_gpt.to_dict()

print("test_trigrams_hum_dict:")
print(list(test_trigrams_hum_dict.items())[0][1][0][:2])

test_trigrams_hum_dict:
('<start>', 'it')


## N-gram frequencies 

Training sets

In [12]:
def calculate_ngram_freq(ngrams):
    ngram_freq = {}
    for ngram_list in ngrams.values():
        for ngram in ngram_list:
            if ngram not in ngram_freq:
                ngram_freq[ngram] = 1
            else:
                ngram_freq[ngram] += 1
    return ngram_freq


def calculate_word_freq(words): 
    word_freq = {}
    for i in range(len(words)):
        for word in words[i]:
            if word not in word_freq:
                word_freq[word] = 1
            else:
                word_freq[word] += 1
    return word_freq


# Calculate n-gram frequencies for human training set
unigram_freq_hum = calculate_word_freq(train_words_hum["Content"])
bigram_freq_hum = calculate_ngram_freq(train_bigrams_hum_dict)
trigram_freq_hum = calculate_ngram_freq(train_trigrams_hum_dict)

# Calculate n-gram frequencies for GPT training set
unigram_freq_gpt = calculate_word_freq(train_words_gpt["Content"])
bigram_freq_gpt = calculate_ngram_freq(train_bigrams_gpt_dict)
trigram_freq_gpt = calculate_ngram_freq(train_trigrams_gpt_dict)

print("Unigram frequencies for human training set:", list(unigram_freq_hum.items())[:5])
print("Bigram frequencies for human training set:", list(bigram_freq_hum.items())[:5])
print("Trigram frequencies for human training set:", list(trigram_freq_hum.items())[:5])
print()
print("Unigram frequencies for GPT training set:", list(unigram_freq_gpt.items())[:5])
print("Bigram frequencies for GPT training set:", list(bigram_freq_gpt.items())[:5])
print("Trigram frequencies for GPT training set:", list(trigram_freq_gpt.items())[:5])

Unigram frequencies for human training set: [('<start>', 35742), ('not', 17576), ('a', 86621), ('matter', 954), ('of', 73932)]
Bigram frequencies for human training set: [(('<start>', 'not'), 197), (('not', 'a'), 863), (('a', 'matter'), 152), (('matter', 'of'), 159), (('of', 'muscles'), 10)]
Trigram frequencies for human training set: [(('<start>', '<start>', 'not'), 197), (('<start>', 'not', 'a'), 16), (('not', 'a', 'matter'), 9), (('a', 'matter', 'of'), 145), (('matter', 'of', 'muscles'), 1)]

Unigram frequencies for GPT training set: [('<start>', 18358), ('when', 11483), ('a', 101329), ('breeze', 10), ('blows', 28)]
Bigram frequencies for GPT training set: [(('<start>', 'when'), 868), (('when', 'a'), 1015), (('a', 'breeze'), 3), (('breeze', 'blows'), 1), (('blows', 'on'), 2)]
Trigram frequencies for GPT training set: [(('<start>', '<start>', 'when'), 868), (('<start>', 'when', 'a'), 139), (('when', 'a', 'breeze'), 1), (('a', 'breeze', 'blows'), 1), (('breeze', 'blows', 'on'), 1)]


In [13]:
print("# of unigrams in the human data:", len(unigram_freq_hum))
print("# of bigrams in the human data:", len(bigram_freq_hum))
print("# of trigrams in the human data:", len(trigram_freq_hum))
print()
print("# of unigrams in the gpt data:", len(unigram_freq_gpt))
print("# of bigrams in the gpt data:", len(bigram_freq_gpt))
print("# of trigrams in the gpt data:", len(trigram_freq_gpt))

# of unigrams in the human data: 57707
# of bigrams in the human data: 799786
# of trigrams in the human data: 2093325

# of unigrams in the gpt data: 38380
# of bigrams in the gpt data: 471878
# of trigrams in the gpt data: 1393225


In [14]:
# Calculate vocabulary 
# (number of unique unigrams in the training set (including both hum and gpt))

combined_unigram_dicts = {**unigram_freq_hum, **unigram_freq_gpt}
vocab = len(combined_unigram_dicts)

Testing sets

Calculate and report the percentage of bigrams/trigrams in the test set (counted with repeats) that do not
appear in the training corpus (this is called the OOV rate). You should calculate two OOV rates, one for
bigrams and one for trigrams.

Compare each bigrams in the testing set with all bigrams in the training set. If they do not equal any of the bigrams in the training set, add 1 to the counter. At the end, divide this number by the total number of bigrams in the testing set

In [15]:
# Combine all of the bigrams in training and all in testing 

In [16]:
def get_oov(test_ngrams, train_ngrams):
    not_in_train = [ngram for ngrams_list in test_ngrams for ngram in ngrams_list if ngram not in train_ngrams]
    oov_rate = len(set(not_in_train)) / sum(len(ngrams_list) for ngrams_list in test_ngrams) 
    return oov_rate

# Combine human and GPT n-grams sets
combined_train_bigrams = train_bigrams_hum.append(train_bigrams_gpt, ignore_index=True)
combined_train_trigrams = train_trigrams_hum.append(train_trigrams_gpt, ignore_index=True)

combined_test_bigrams = test_bigrams_hum.append(test_bigrams_gpt, ignore_index=True)
combined_test_trigrams = test_trigrams_hum.append(test_trigrams_gpt, ignore_index=True)

# Calculate OOV rates
oov_rate_combined_bigrams = get_oov(combined_test_bigrams, combined_train_bigrams)
oov_rate_combined_trigrams = get_oov(combined_test_trigrams, combined_train_trigrams)

print("Combined OOV rate for bigrams:", oov_rate_combined_bigrams)
print("Combined OOV rate for trigrams:", oov_rate_combined_trigrams)


/var/folders/72/9576tn712d71pv50d2f870g40000gn/T/ipykernel_9373/3018787587.py:7: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_train_bigrams = train_bigrams_hum.append(train_bigrams_gpt, ignore_index=True)
/var/folders/72/9576tn712d71pv50d2f870g40000gn/T/ipykernel_9373/3018787587.py:8: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_train_trigrams = train_trigrams_hum.append(train_trigrams_gpt, ignore_index=True)
/var/folders/72/9576tn712d71pv50d2f870g40000gn/T/ipykernel_9373/3018787587.py:10: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_test_bigrams = test_bigrams_hum.append(test_bigrams_gpt, ignore_index=True)
/var/folders/72/9576tn712d71pv50d2f870g40000gn/T/ipykernel_9373/3018787587.py:11: Fu

Combined OOV rate for bigrams: 0.30536979636477
Combined OOV rate for trigrams: 0.6719696959942729


# Using n-gram models to predict whether a document is human or GPT generated

Find bigram probabilities 
- classify each document using bigrams
- classify each document using trigrams 

In [17]:
# BIGRAM

# go through each document in the human and then in the gpt TEST sets
# for each document, calculate the probability of it being human generated, using human counts and P(human) 
# and then ALSO calculate the probability of it being gpt generated using gpt counts and P(GPT)
# classify based on higher probability 

p_hum = 35742/54100 
p_gpt = 18358/54100

def classify_bi(document):
    pi_hum = 0
    pi_gpt = 0
    
    for i in range(len(document)):
        if document[i] not in bigram_freq_hum:
            bigram_freq_hum[document[i]] = 0
        if document[i][0] not in unigram_freq_hum:
            unigram_freq_hum[document[i][0]] = 0
            
        pi_hum += np.log((bigram_freq_hum[document[i]] + 1) / (unigram_freq_hum[document[i][0]] + vocab))
        
        if document[i] not in bigram_freq_gpt:
            bigram_freq_gpt[document[i]] = 0
        if document[i][0] not in unigram_freq_gpt:
            unigram_freq_gpt[document[i][0]] = 0
          
        pi_gpt += np.log((bigram_freq_gpt[document[i]] + 1) / (unigram_freq_gpt[document[i][0]] + vocab))
    
    human_prob = np.log(p_hum) + pi_hum
    gpt_prob = np.log(p_gpt) + pi_gpt
        
    if human_prob > gpt_prob:
        return 0    # classified as human
    elif gpt_prob > human_prob:
        return 1    # classified as gpt
    
count = 0
correct = 0
    
# For human set
for key, value in test_bigrams_hum_dict.items():
    count += 1
    test_bigrams_hum_dict[key].append(('classification', classify_bi(value)))
    if classify_bi(value) == 0:
        correct += 1
        
# For gpt set
for key, value in test_bigrams_gpt_dict.items():
    count += 1
    test_bigrams_gpt_dict[key].append(('classification', classify_bi(value)))
    if classify_bi(value) == 1:
        correct += 1

print("Bigram accuracy:", correct/count)

Bigram accuracy: 0.9595741141241058


In [18]:
# TRIGRAM

p_hum = 35742/54100 
p_gpt = 18358/54100

def classify_tri(document):
    pi_hum = 0
    pi_gpt = 0
    
    for i in range(len(document)):
        if document[i] not in trigram_freq_hum:
            trigram_freq_hum[document[i]] = 0
        if document[i][0] not in bigram_freq_hum:
            bigram_freq_hum[document[i][:2]] = 0
            
        pi_hum += np.log((trigram_freq_hum[document[i]] + 1) / (bigram_freq_hum[document[i][:2]] + vocab))
        
        if document[i] not in trigram_freq_gpt:
            trigram_freq_gpt[document[i]] = 0
        if document[i][0] not in bigram_freq_gpt:
            bigram_freq_gpt[document[i][:2]] = 0
          
        pi_gpt += np.log((trigram_freq_gpt[document[i]] + 1) / (bigram_freq_gpt[document[i][:2]] + vocab))
    
    human_prob = np.log(p_hum) + pi_hum
    gpt_prob = np.log(p_gpt) + pi_gpt
        
    if human_prob > gpt_prob:
        return 0    # classified as human
    elif gpt_prob > human_prob:
        return 1    # classified as gpt
    
count2 = 0
correct2 = 0
    
# For human set
for key, value in test_trigrams_hum_dict.items():
    count2 += 1
    test_trigrams_hum_dict[key].append(('classification', classify_tri(value)))
    if classify_tri(value) == 0:
        correct2 += 1
        
# For gpt set
for key, value in test_trigrams_gpt_dict.items():
    count2 += 1
    test_trigrams_gpt_dict[key].append(('classification', classify_tri(value)))
    if classify_tri(value) == 1:
        correct2 += 1
        
print("Trigram accuracy:", correct2/count2)

Trigram accuracy: 0.9422725004159042


## Text generation

In [20]:
# Human Bigram

print("Text generated from human bigrams:")
for j in range(5):
    initial = '<start>'
    sent = []

    for k in range(20):
        selected_bigrams = {}

        # Iterate over the items in bigrams and filter based on the condition key[0] == initial
        for key, value in bigram_freq_hum.items():
            if key[0] == initial:
                selected_bigrams[key] = value

        new_dict = {}

        # Iterate over the items in bigrams and calculate exp(count/temperature)
        for bigram, count in selected_bigrams.items():
            new_dict[bigram] = np.exp(count / 50)

        # Initialize a variable for denominator and set it to zero
        denom = 0

        # Iterate over the values in new_dict and sum them to calculate the denominator
        for value in new_dict.values():
            denom += value

        # Initialize an empty dictionary for final_dict
        final_dict = {}

        # Iterate over the items in new_dict and calculate value/denominator
        for bigram, value in new_dict.items():
            final_dict[bigram] = value / denom

        second_elements = []
        for key in final_dict.keys():
            second_element = key[1]
            second_elements.append(second_element) 

        initial = np.random.choice(second_elements, 1, p = list(final_dict.values()))   
        sent.append(str(initial))

        if initial == '<end>':
            break
    print(j+1, ":", sent)

Text generated from human bigrams:
1 : ["['ideas']", "['intersect']", "['because']", "['motion']", "['picture']", "['that']", "['poke']", "['a']", "['freshwater']", "['from']", "['sams']", "['wages']", "['management']", "['body']", "['occasionally']", "['he']", "['reveres']", "['could']", "['that']", "['causes']"]
2 : ["['deutschland']", "['translates']", "['as']", "['web']", "['savvy']", "['desktop']", "['so']", "['6']", "['such']", "[',']", "['138']", "[',']", "['purpose']", "['<end>']"]
3 : ["['ianal']", "['i']", "['entitled']", "['to']", "['transpose']", "['lyrics']", "['did']", "['fairly']", "['arduous']", "['terms']", "['despite']", "['this']", "['produces']", "['antiinflammatory']", "['medication']", "['dissolve']", "['or']", "['turning']", "['circles']", "['that']"]
4 : ["['several']", "['exponentially']", "['increased']", "['regulation']", "['groups']", "['possibly']", "['blockades']", "['enforced']", "['when']", "['technical']", "['cause']", "['calluses']", "['and']", "['swel

In [78]:
# GPT Bigram

print("Text generated from GPT bigrams:")
for j in range(5):
    initial = '<start>'
    sent = []

    for k in range(20):
        selected_bigrams = {}

        # Iterate over the items in bigrams and filter based on the condition key[0] == initial
        for key, value in bigram_freq_gpt.items():
            if key[0] == initial:
                selected_bigrams[key] = value

        new_dict = {}

        # Iterate over the items in bigrams and calculate exp(count/temperature)
        for bigram, count in selected_bigrams.items():
            new_dict[bigram] = np.exp(count / 50)

        # Initialize a variable for denominator and set it to zero
        denom = 0

        # Iterate over the values in new_dict and sum them to calculate the denominator
        for value in new_dict.values():
            denom += value

        # Initialize an empty dictionary for final_dict
        final_dict = {}

        # Iterate over the items in new_dict and calculate value/denominator
        for bigram, value in new_dict.items():
            final_dict[bigram] = value / denom

        second_elements = []
        for key in final_dict.keys():
            second_element = key[1]
            second_elements.append(second_element) 

        initial = np.random.choice(second_elements, 1, p = list(final_dict.values()))   
        sent.append(str(initial))

        if initial == '<end>':
            break
    print(j+1, ":", sent)

Text generated from GPT bigrams:
1 : ["['beverly']", "['hills']", "['very']", "['positive']", "['thing']", "['that']", "['reasonable']", "['grounds']", "['to']", "['urban']", "['cultures']", "['themselves']", "['that']", "['wo']", "['nt']", "['evaporate']", "['more']", "['values']", "['may']", "['simply']"]
2 : ["['siamese']", "['twins']", "['will']", "['fit']", "['both']", "['great']", "['philosophers']", "['play']", "['quickly']", "['getting']", "['hepatitis']", "['depend']", "['on']", "['screen']", "['has']", "['nuclear']", "['explosions']", "['are']", "['encountered']", "['in']"]
3 : ["['natural']", "['urges']", "['to']", "['sample']", "['and']", "['catastrophic']", "['consequences']", "['on']", "['chemical']", "['peels']", "[',']", "['pasteurized']", "[',']", "['502']", "['in']", "['boxes']", "['could']", "['escape']", "['keys']", "['available']"]
4 : ["['hubert']", "['l']", "['sound']", "['1']", "['100']", "['41']", "['0']", "['1']", "['kilograms']", "['.']", "['<end>']"]
5 : ["[